# Mini-projet n°2 - Conditional VAEs

L'objectif de ce projet est de définir et entraîner un Conditional VAE (CVAE). Nous allons tout d'abord définir ce qu'est un CVAE et la différence avec un VAE, puis définir et entraîner un CVAE sur le dataset Fashin-MNIST.

## Définition d'un CVAE

## Création du CVAE

## Entraînement du CVAE

## Création de 5 échantillons de chaque classe 